In [4]:
#import packages
import numpy as np
import cv2
import pandas as pd
import matplotlib.pylab as plt
import os
from os.path import join

In [5]:
#read the answer file
img_ans = cv2.imread(os.getcwd() + '\\Answer\\3A.png') 

#read image of one of the students
img_stu = cv2.imread(os.getcwd() + '\\Data\\2000113_NguyenBaoNgan_3A.png') 

ans = ['A','B','C','D','E']
st,ed = 160, 270
step = 139

lst, name = [], []
path = os.getcwd() + '\\Data'

#split the name, surname, student ID and the answer's code
for i in os.listdir(path):
    s = i[0: -4].split('_')
    t = 0
    pos, temp = [], []
    for j,k in enumerate(s[1]+'A'): 
        if k.isupper(): pos.append(j)

    for d in range(len(pos)-1): 
        temp.append( s[1][pos[d] : pos[d+1]] )
    name.append(temp)
    lst.append(s)

In [38]:
#setting up the student dataframe and export it
df = pd.DataFrame(lst, columns = ['StudentID', 'Name', 'Code'])
df['Surname'] = [ ' '.join(i[0: len(i)-1]) for i in name ]
df['First Name'] = [ i[-1] for i in name ]
df = df[['StudentID', 'Surname', 'First Name', 'Code']]
df.to_csv(os.getcwd() + '\\Output\\student.csv',index = False, header = True)
print("2. From the Data folder, generating student.csv (StudentID, Surname, Firstname, Code) by coding")
df.index += 1
df

2. From the Data folder, generating student.csv (StudentID, Surname, Firstname, Code) by coding


,StudentID,Surname,First Name,Code
1,2000101,Nguyen Van,An,3A
2,2000102,Nguyen Truong,Bao,3A
3,2000103,Tran Van,Cuong,3A
4,2000104,Truong Thi,Diem,3A
5,2000105,Le Thanh,Em,3A
6,2000106,Truong Thi,Fuc,3A
7,2000107,Nguyen Thi,Giang,3A
8,2000108,Tran Minh,Hang,3A
9,2000109,Huynh Cong,Jack,3A
10,2000110,Nguyen Trung,Kim,3A


In [7]:
#Grayscaling the image for better detection
def grayscale(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    #blur = cv2.GaussianBlur(gray, (5,5), 0)
    _, thresh = cv2.threshold(gray, 170, 255, cv2.THRESH_BINARY_INV)
    return thresh

In [8]:
#split the image into 5 different sections for later calculations
def chunk_process(image): 
    he, wi = image.shape[0], image.shape[1]
    hedv = he // 5
    boxes = [[],[],[],[],[]]
    t1 = 0
    for i in range(0, 5):
        row = image[t1 : hedv, :]
        hedv += he // 5
        t1 += he // 5
        t2 = 0
        widv = wi // 5
        for _ in range(0,5):
            col = row[:, t2 : widv]
            widv += wi // 5
            t2 += wi // 5
            boxes[i].append(col)
    return boxes

In [9]:
#Split the sections again into 25 boxes and calculating the answer
def main_process(arr, img, x, y, z):
    d = z
    for mtp in range(0,6):
        pic = grayscale(img[st + step * mtp: ed + step * mtp, x : y])
        box = chunk_process(pic)
        for i in range(0,5):
            d += 1
            max = -999
            for j in range(0,5):
                temp = cv2.countNonZero(box[i][j])
                if max < temp: 
                    max = temp
                    idx = j
            arr.append(ans[idx])
    return d

In [10]:
#setting up the answers list
answer = []
d = main_process(answer, img_ans, 140, 335, 0)
_ = main_process(answer, img_ans, 500, 690, d)

In [11]:
#print first 5 answer of a random student
stu = []
d = main_process(stu,img_stu,140, 335, 0)
main_process(stu,img_stu,500, 690, d)
print("3. Generating the first 5 answers of one student")
print([stu[i] for i in range(0,5)])

3. Generating the first 5 answers of one student
['A', 'D', 'B', 'E', 'C']


In [12]:
print("4. Generating all answers of one student")
print(stu)

4. Generating all answers of one student
['A', 'D', 'B', 'E', 'C', 'B', 'D', 'C', 'A', 'D', 'C', 'B', 'A', 'A', 'D', 'D', 'B', 'D', 'A', 'C', 'A', 'D', 'C', 'D', 'A', 'E', 'D', 'C', 'B', 'A', 'C', 'D', 'C', 'A', 'E', 'B', 'E', 'A', 'C', 'B', 'E', 'C', 'A', 'B', 'E', 'E', 'B', 'A', 'C', 'A', 'D', 'C', 'E', 'A', 'C', 'C', 'B', 'E', 'A', 'D']


In [13]:
#counting the scores of the students
sid, score, pf = [], [], []
num = [ _ for _ in range(1,61) ]
wrong = [ 0 for _ in range(0,60) ]

for i in os.listdir(path):
    sid.append(i[0:7])
    input_stu = cv2.imread(join(path, i)) #read the students' image files
    stu = []
    temp = 0

    d = main_process(stu, input_stu, 140, 335, 0)
    main_process(stu, input_stu, 500, 690, d)

    for x in range(0, len(stu)):
        if stu[x] == answer[x]:
            temp += 1
        else: wrong[x] += 1
            
    if temp <= 30: pf.append('Fail')
    else: pf.append('Pass')

    score.append("{:.2f}".format((temp / 60) * 10))

In [35]:
#Create dataframe for ranking the questions' difficulty
question = pd.DataFrame(num, columns = ['Question'])
question['Count'] = wrong
question = question.sort_values(by='Count')
res = question['Question'].tolist()
print("6. Summary which 3 questions are the most difficult.")
for i in range(0,3):
    print(i+1,'-', res[i])

6. Summary which 3 questions are the most difficult.
1 - 6
2 - 7
3 - 56
